In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import os
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


In [4]:
# Cargar dataset desde CSV
csv_path = "dataset_labels.csv"
df = pd.read_csv(csv_path)

# Codificar etiquetas numéricamente
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])
num_classes = len(label_encoder.classes_)

# Dividir en train, val y test
train_df = df[df["split"] == "train"]
val_df = df[df["split"] == "val"]
test_df = df[df["split"] == "test"]

# Función para cargar imágenes
def load_images(df, img_size=(299, 299)):  # InceptionV3 usa 299x299
    images = []
    labels = []
    for _, row in df.iterrows():
        img = cv2.imread(row["image_path"])
        img = cv2.resize(img, img_size)
        img = img.astype("float32") / 255.0  # Normalizar
        images.append(img)
        labels.append(row["label"])
    return np.array(images), to_categorical(labels, num_classes)

# Cargar imágenes
X_train, y_train = load_images(train_df)
X_val, y_val = load_images(val_df)
X_test, y_test = load_images(test_df)  # Agregado para testeo

# Cargar modelo preentrenado InceptionV3
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(299, 299, 3))
base_model.trainable = False  # Congelar capas preentrenadas

# Construcción del modelo
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=outputs)

# Compilar el modelo
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [6]:
# Entrenar el modelo
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)

# Evaluar el modelo en el conjunto de testeo
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calcular métricas
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

# Matriz de confusión y reporte de clasificación
conf_matrix = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred)

# Imprimir resultados
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)

Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - accuracy: 0.3881 - loss: 1.3425 - val_accuracy: 0.6691 - val_loss: 0.8854
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.6287 - loss: 0.8994 - val_accuracy: 0.6957 - val_loss: 0.7687
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 104s 2s/step - accuracy: 0.6878 - loss: 0.7304 - val_accuracy: 0.7101 - val_loss: 0.7183
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.7323 - loss: 0.6781 - val_accuracy: 0.7391 - val_loss: 0.6847
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.7518 - loss: 0.6369 - val_accuracy: 0.7415 - val_loss: 0.6686
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step
Accuracy: 0.6154
Precision: 0.6855
Recall: 0.6154
F1-score: 0.6115
Confusion Matrix:
 [[42  7  1  0]
 [28 21  1  0]
 [15  6 23  6]
 [ 4  2  0 26]]
Classification Report:
               precision    recall  f1-score   support

           0       0.47      0.84      0.60        50
           1       0.58      0.42      0.49        50
 